In [ ]:
import itertools  # 반복문과 조합 관련 유틸리티
import os  # 운영체제 파일 경로 등 처리
import os.path as osp  # 경로(join, split 등) 처리 단축
import time  # 시간 측정 및 대기
from collections import OrderedDict, defaultdict  # 순서 있는 dict, 기본값 딕셔너리
from datetime import datetime  # 날짜/시간 객체
from tqdm import tqdm  # 진행 바 표시

import h5py  # HDF5 파일 입출력

import torch  # PyTorch 텐서 연산
import collections  # 컬렉션 관련 타입
import tensorflow_datasets as tfds  # TFDS 데이터셋 로드
import numpy as np  # 수치 연산
import tkinter as tk  # 간단한 GUI (언어 입력 등)
from tkinter import simpledialog  # 간단한 입력 다이얼로그
from PIL import Image, ImageTk  # 이미지 처리 및 표시
import argparse  # 명령행 인자 파싱

from IPython.display import display, Image as IPyImage
def get_image_list_np(img_rgb_dir_path, remove_index_list):
    """
    지정한 디렉토리에서 RGB 이미지 파일 목록을 불러와 numpy 배열로 반환합니다.
    remove_index_list에 포함된 인덱스는 건너뜁니다.
    """
    cur_camera_rgb_list = []  # 개별 프레임 저장할 리스트
    img_name_list = os.listdir(img_rgb_dir_path)  # 디렉토리 내 파일 이름 목록
    img_name_list = sorted(img_name_list)  # 정렬하여 순서 보장

    for idx, img_name in enumerate(img_name_list):  # 각 이미지 파일에 대해
        if idx in remove_index_list:  # 제거할 인덱스면 건너뜀
            continue

        img_path = os.path.join(img_rgb_dir_path, img_name)  # 전체 경로 생성

        # (w 640, h 480)
        img_frame = Image.open(img_path).convert('RGB')  # PIL로 열어 RGB로 변환
        img_np = np.array(img_frame)  # numpy 배열로 변환 (H,W,3)
        cur_camera_rgb_list.append(img_np)  # 리스트에 추가

    cur_camera_rgb_np = np.array(cur_camera_rgb_list)  # 전체 시퀀스 배열화
    print('+++++++++++++++')
    print(f"img_rgb_dir_path: {img_rgb_dir_path}")  # 경로 정보 출력
    print(f'cur_camera_rgb_np size: {cur_camera_rgb_np.shape}')  # 배열 크기 출력

    return cur_camera_rgb_np  # 반환


def plot_smooth_action(traj_act_xyz_np, fig_name):
    """
    예측된 행동 값(x,y,z)에 대해 부드러운 플롯을 그리고 저장합니다.
    """
    import matplotlib.pyplot as plt  # 플롯용 라이브러리
    plt.figure(figsize=(12, 4))  # 도표 크기 설정
    figure_name = ["x", "y", "z"]  # 축 라벨
    for i in range(3):  # 각 축별로 subplot 생성
        plt.subplot(1, 3, i + 1)
        plt.plot(range(traj_act_xyz_np.shape[0]), traj_act_xyz_np[:, i], label='cur_action')  # 선 그래프
        plt.title(figure_name[i])  # 제목 설정
        plt.legend()  # 범례 표시
    plt.suptitle(f"Differences between predicted and target actions_traj")  # 전체 제목
    plt.tight_layout()  # 레이아웃 자동 조정

    work_dir = '/home/parkjeongsu/TinyVLA_pre/droid_traj'  # 저장 디렉토리 기본 경로
    figure_dir_path = os.path.join(work_dir, f"smooth_action_results")  # 하위 폴더 경로
    os.makedirs(figure_dir_path, exist_ok=True)  # 디렉토리 생성(이미 있으면 무시)
    figure_path = os.path.join(figure_dir_path, f"{fig_name}.png")  # 파일 경로
    plt.savefig(figure_path)  # 이미지 저장
    plt.clf()  # 현재 figure 초기화


def print_h5_structure(group, indent=0):
    """
    HDF5 파일 그룹 구조를 재귀적으로 출력합니다.
    """
    for name in group:
        item = group[name]
        print(" " * indent + f"name: {name}")
        if isinstance(item, h5py.Group):  # 그룹일 경우
            print(" " * indent + f"Group: {name}")
            print_h5_structure(item, indent + 2)  # 재귀 호출
        elif isinstance(item, h5py.Dataset):  # 데이터셋일 경우
            print(" " * indent + f"Dataset: {name} (Shape: {item.shape}, Dtype: {item.dtype})")
        else:
            print(" " * indent + f"Unknown item: {name}")


def print_dict_structure(cur_dict, indent=0):
    """
    Python dict 구조를 재귀적으로 출력합니다.
    """
    for name in cur_dict.keys():
        item = cur_dict[name]
        print(" " * indent + f"name: {name}")
        if isinstance(item, dict):
            print(" " * indent + f"Dict: {name}")
            print_dict_structure(item, indent + 2)
        elif isinstance(item, np.ndarray):
            print(" " * indent + f"Array: {name} (Shape: {item.shape}, Dtype: {item.dtype})")
        else:
            print(" " * indent + f"Unknown item: {name}")


def to_numpy(x):
    """
    중첩된 torch.Tensor를 재귀적으로 numpy 배열로 변환합니다.
    """
    def f(tensor):
        if tensor.is_cuda:
            return tensor.detach().cpu().numpy()
        else:
            return tensor.detach().numpy()

    return recursive_dict_list_tuple_apply(
        x,
        {
            torch.Tensor: f,
            np.ndarray: lambda x: x,
            type(None): lambda x: x,
        }
    )


def recursive_dict_list_tuple_apply(x, type_func_dict):
    """
    중첩된 dict/list/tuple 내부 요소에 주어진 함수를 적용합니다.
    """
    assert (list not in type_func_dict)
    assert (tuple not in type_func_dict)
    assert (dict not in type_func_dict)

    if isinstance(x, (dict, collections.OrderedDict)):
        new_x = collections.OrderedDict() if isinstance(x, collections.OrderedDict) else dict()
        for k, v in x.items():
            new_x[k] = recursive_dict_list_tuple_apply(v, type_func_dict)
        return new_x
    elif isinstance(x, (list, tuple)):
        ret = [recursive_dict_list_tuple_apply(v, type_func_dict) for v in x]
        return tuple(ret) if isinstance(x, tuple) else ret
    else:
        for t, f in type_func_dict.items():
            if isinstance(x, t):
                return f(x)
        return x  # 그 외 타입은 그대로 반환


def matrix_to_rotation_6d(matrix: torch.Tensor) -> torch.Tensor:
    """
    3x3 회전 행렬을 6D 표현으로 변환합니다 (첫 두 열 사용).
    """
    batch_dim = matrix.size()[:-2]
    return matrix[..., :2, :].clone().reshape(batch_dim + (6,))


def euler_angles_to_rot_6d(euler_angles, convention="XYZ"):
    """
    Euler 각도를 6D 회전 표현으로 변환합니다.
    """
    rot_mat = euler_angles_to_matrix(euler_angles, convention="XYZ")
    return matrix_to_rotation_6d(rot_mat)


def _axis_angle_rotation(axis: str, angle: torch.Tensor) -> torch.Tensor:
    """
    X/Y/Z 축을 기준으로 회전 행렬을 생성합니다.
    """
    cos = torch.cos(angle)
    sin = torch.sin(angle)
    one = torch.ones_like(angle)
    zero = torch.zeros_like(angle)

    if axis == "X":
        R_flat = (one, zero, zero, zero, cos, -sin, zero, sin, cos)
    elif axis == "Y":
        R_flat = (cos, zero, sin, zero, one, zero, -sin, zero, cos)
    elif axis == "Z":
        R_flat = (cos, -sin, zero, sin, cos, zero, zero, zero, one)
    else:
        raise ValueError("letter must be either X, Y or Z.")

    return torch.stack(R_flat, -1).reshape(angle.shape + (3, 3))


def euler_angles_to_matrix(euler_angles: torch.Tensor, convention: str) -> torch.Tensor:
    """
    Euler 각도를 3x3 회전 행렬로 변환합니다.
    """
    if euler_angles.dim() == 0 or euler_angles.shape[-1] != 3:
        raise ValueError("Invalid input euler angles.")
    if len(convention) != 3:
        raise ValueError("Convention must have 3 letters.")
    if convention[1] in (convention[0], convention[2]):
        raise ValueError(f"Invalid convention {convention}.")
    for letter in convention:
        if letter not in ("X", "Y", "Z"):
            raise ValueError(f"Invalid letter {letter} in convention string.")
    matrices = [
        _axis_angle_rotation(c, e) for c, e in zip(convention, torch.unbind(euler_angles, -1))
    ]
    return torch.matmul(torch.matmul(matrices[0], matrices[1]), matrices[2])


def convert_h5py2np_dict(group, state_np_dict, indent=0):
    """
    HDF5 그룹을 순회하며 numpy dict로 변환하고 구조 출력합니다.
    """
    for name in group:
        item = group[name]
        print(" " * indent + f"name: {name}")
        if isinstance(item, h5py.Group):
            state_np_dict[name] = {}
            convert_h5py2np_dict(item, state_np_dict[name], indent + 2)
        elif isinstance(item, h5py.Dataset):
            state_np_dict[name] = item[...]
            print(" " * indent + f"Dataset: {name} (Shape: {item.shape}, Dtype: {item.dtype})")
        else:
            state_np_dict[name] = item


def print_name(name):
    """단순히 이름을 출력"""
    print(name)

def generate_h5(obs_replay, action_replay, cfg, total_traj_cnt, act_root_dir_path, edit_flag):
    """
    주어진 관측(obs_replay)과 행동(action_replay)을 이용해 HDF5 파일로 저장합니다.
    - obs_replay: {'qpos', 'qvel', 'images': {cam1, cam2, ...}}
    - cfg: 카메라 이름, 이미지 크기, 상태/행동 차원, 언어 지시문 등 설정
    """
    data_dict = {
        '/observations/qpos': obs_replay['qpos'],  # 위치 정보
        '/observations/qvel': obs_replay['qvel'],  # 속도 정보
        '/action': action_replay,  # 행동 벡터
        'is_edited': np.array(edit_flag)  # 편집 플래그
    }
    # 이미지 데이터 추가
    for cam_name in cfg['camera_names']:
        data_dict[f'/observations/images/{cam_name}'] = obs_replay['images'][cam_name]

    max_timesteps = len(data_dict['/observations/qpos'])
    print(f'max_timesteps: {max_timesteps}')

    data_dir = act_root_dir_path  # 저장 디렉토리
    dataset_path = os.path.join(data_dir, f'episode_{total_traj_cnt}')
    with h5py.File(dataset_path + '.hdf5', 'w', rdcc_nbytes=1024**2*2) as root:
        root.attrs['sim'] = True  # 시뮬레이션 메타데이터
        obs = root.create_group('observations')
        image = obs.create_group('images')
        # 카메라별 이미지 데이터셋 생성
        for cam_name in cfg['camera_names']:
            image.create_dataset(cam_name,
                (max_timesteps, cfg['cam_height'], cfg['cam_width'], 3),
                dtype='uint8', chunks=(1, cfg['cam_height'], cfg['cam_width'], 3))
        # 상태 데이터셋 생성
        obs.create_dataset('qpos', (max_timesteps, cfg['state_dim']))
        obs.create_dataset('qvel', (max_timesteps, cfg['state_dim']))
        # 행동 데이터셋 생성
        root.create_dataset('action', (max_timesteps, cfg['action_dim']))
        root.create_dataset('is_edited', (1,))
        # 언어 지시문 저장
        root.create_dataset("language_raw", data=[cfg['lang_intrs']])

        print(f'==== generate h5 ======')
        # 실제 데이터 쓰기
        for name, array in data_dict.items():
            print(f"name: {name}, array: {array.shape}")
            root[name][...] = array

user_input = None  # GUI 입력용 전역 변수

def show_gif(images):
    """이미지 리스트를 GIF로 저장하여 보여줍니다."""
    # 폴더 + 파일명(.gif)로 저장 경로 지정
    path = os.path.join('/home/parkjeongsu/TinyVLA_pre/droid_image', 'result.gif')
    images[0].save(path, save_all=True, append_images=images[1:], duration=int(1000/15), loop=0)
    display(IPyImage(filename=path))


# Jupyter 환경에선 argparse 대신 직접 지정
# ✅ 사용자 정의 설정 (Jupyter용 경로)
class Args:
    pass

args = Args()
args.src_root = "/home/parkjeongsu/TinyVLA/Droid"  # ✅ TFDS 경로
args.name = "droid_100"  # ✅ TFDS 데이터셋 이름

# ✅ HDF5 저장 경로 구성
task_name = "droid_1dot7t_lang"
act_target_root = os.path.join(args.src_root, "droid_with_lang")
os.makedirs(act_target_root, exist_ok=True)

# ✅ 서브디렉토리명: smoothing 정보 포함
act_root_dir_name = f"{task_name}_succ_t0001_s-0-0"
act_root_dir_path = os.path.join(act_target_root, act_root_dir_name)
os.makedirs(act_root_dir_path, exist_ok=True)


# ✅ cfg 설정 (저장 파라미터 포함)
cfg = {
    "task_name": task_name,
    "camera_names": ["left", "right", "wrist"],  # ✅ wrist 포함
    "dataset_dir": act_target_root,
    "cam_height": 180,
    "cam_width": 320,
    "state_dim": 7,
    "action_dim": 7,  # ✅ rot6D 제거로 7차원 사용
    "lang_intrs": "close the lid of the box",
}
# ✅ TFDS 로드
print("Loading TFDS dataset...")
ds = tfds.load(args.name, data_dir=args.src_root, split="train")

# ✅ 경로 확인 출력
print("✅ TFDS 경로:", os.path.join(args.src_root, args.name))
print("✅ 저장 경로:", act_root_dir_path)

raw_lang = cfg['lang_intrs']  # 원시 언어 지시문
print('raw_lang:', raw_lang)

#parser = argparse.ArgumentParser()  # 명령행 파서 생성
#parser.add_argument("/home/parkjeongsu/TinyVLA_pre/Droid", default='/media/rl/PSSD-6')  # TFDS 루트 경로 : parser.add_argument("--src_root", default='/media/rl/PSSD-6')
#parser.add_argument("/home/parkjeongsu/TinyVLA_pre/Droid/droid_100", default="droid")  # 데이터셋 이름
#args = parser.parse_args()

# 출력 디렉토리 생성
act_target_root = os.path.join(args.src_root, "droid_with_lang")
os.makedirs(act_target_root, exist_ok=True)

smooth_action = False  # 부드러운 행동 처리 여부
smooth_order = 0  # 부드러운 순서
smooth_window_size = 0  # 부드러운 윈도우 크기

framework = 'droid'  # 프레임워크 종류
act_pos_thres = 0.001  # 행동 임계값

# 저장 경로 설정
act_root_dir_name = f'{cfg["task_name"]}_succ_t0001_s-0-0'
act_root_dir_path = os.path.join(act_target_root, act_root_dir_name)
os.makedirs(act_root_dir_path, exist_ok=True)

# tfds 데이터셋 로드
ds = tfds.load(args.name, data_dir=args.src_root, split="train")

# TFDS 로드 및 HDF5 변환 루프

# 카메라 이름과 이미지 파일명을 매핑하기 위한 빈 딕셔너리 초기화
IMAGE_NAME_TO_CAM_KEY_MAPPING = dict()

# 성공/실패 및 전체 트래젝토리 카운트를 저장할 변수 초기화
succ_traj_count = 0    # 성공한 에피소드 수
fail_traj_count = 0    # 실패한 에피소드 수
total_traj_cnt = 0     # 전체 처리한 에피소드 수

# 액션 값의 최대/최소, 정규화 통계, 전체 상태 딕셔너리 초기화
max_action_np = None                      # 액션 데이터 중 최대값 저장용
min_action_np = None                      # 액션 데이터 중 최소값 저장용
data_normalize_stats = dict()             # 정규화 통계(평균·표준편차 등) 저장용
all_traj_state_total_np_dict = dict()     # 모든 트래젝토리 상태 데이터를 모아둘 딕셔너리

# TFDS 데이터셋 로드 (droid 이름, 로컬 경로, train 분할)
ds = tfds.load(args.name, data_dir=args.src_root, split="train")

# 각 에피소드를 순회하며 HDF5로 변환
for episode in tqdm(ds):
    if os.path.exists(os.path.join(act_root_dir_path, f'episode_{total_traj_cnt}.hdf5')):
        total_traj_cnt += 1
        continue

    state_total_np_dict = dict()
    cur_actions = []
    cur_obs_image = {'1': [], '2': [], 'wrist': []}
    cur_obs_gripper_pos = []
    cur_obs_joint_state = []
    cur_obs_cartesian_position = []
    raw_lang = ""
    cur_actions_dict = {}
    edit_flag = 0

    for idx, step in enumerate(episode['steps']):
        lang1 = step['language_instruction'].numpy().decode('utf-8').strip()
        lang2 = step['language_instruction_2'].numpy().decode('utf-8').strip()
        lang3 = step['language_instruction_3'].numpy().decode('utf-8').strip()

        if all(len(lang) < 4 for lang in [lang1, lang2, lang3]):
            print("No valid language instruction in this episode....")
            edit_flag = 1
            break  # ✅ 모든 스텝이 이럴 가능성 고려해서 break 추가
        else:
            # ✅ 언어 선택은 한 번만 하면 되므로, idx==0에서만 수행
            if idx == 0:
                raw_lang = lang1 if len(lang1) >= 4 else (lang2 if len(lang2) >= 4 else lang3)

    # 🔽 전체 에피소드 스킵 조건
    if edit_flag == 1:
        total_traj_cnt += 1
        continue


    # 🔽 3. 실제 step 처리 루프
    for idx, step in enumerate(episode['steps']):
        if idx == 0:
            cur_actions_dict = {k: [] for k in step['action_dict'].keys()}

        cur_actions.append(step['action'].numpy())
        cur_obs_image['1'].append(step['observation']['exterior_image_1_left'].numpy())
        cur_obs_image['2'].append(step['observation']['exterior_image_2_left'].numpy())
        cur_obs_image['wrist'].append(step['observation']['wrist_image_left'].numpy())
        cur_obs_gripper_pos.append(step['observation']['gripper_position'].numpy())
        cur_obs_joint_state.append(step['observation']['joint_position'].numpy())
        raw_lang = step['language_instruction'].numpy().decode('utf-8')
        cur_obs_cartesian_position.append(step['observation']['cartesian_position'].numpy())

        for k, v in cur_actions_dict.items():
            cur_actions_dict[k].append(step['action_dict'][k].numpy())

    if idx == 0 or edit_flag == 1:
        all_images_np = np.concatenate((np.array(cur_obs_image['1']), np.array(cur_obs_image['2'])), axis=2)
        all_images = [Image.fromarray(each) for each in all_images_np]
        show_gif(all_images)
        raw_lang = input("please write a language instruction:")
        print(f"Your input: {raw_lang}")

    cur_obs_image = {k: np.array(v) for k, v in cur_obs_image.items()}
    cur_obs_cartesian_position = torch.from_numpy(np.array(cur_obs_cartesian_position))

    state_total_np_dict['obs_cartesian_position'] = cur_obs_cartesian_position
    state_total_np_dict['obs_gripper_position'] = torch.from_numpy(np.array(cur_obs_gripper_pos))
    state_total_np_dict['obs_joint_positions'] = torch.from_numpy(np.array(cur_obs_joint_state))

    traj_len = min(len(cur_obs_image['1']), len(cur_obs_image['2']), len(cur_obs_image['wrist']))
    print(f'traj_len: {traj_len}')

    traj_actions = np.array(cur_actions)[:traj_len]  # ✅ 그대로 사용
    print(f"traj_actions shape: {traj_actions.shape}")

    traj_qpos = np.concatenate((state_total_np_dict['obs_cartesian_position'], state_total_np_dict['obs_gripper_position']), axis=-1)[:traj_len]
    traj_qvel = np.zeros_like(traj_qpos)
    print(f"traj_qpos shape: {traj_qpos.shape}")
    print(f"traj_qvel shape: {traj_qvel.shape}")

    left_imgs = cur_obs_image['1'][:traj_len]
    right_imgs = cur_obs_image['2'][:traj_len]
    wrist_imgs = cur_obs_image['wrist'][:traj_len]  # ✅ wrist
    print(f"left_imgs shape: {left_imgs.shape}")
    print(f"right_imgs shape: {right_imgs.shape}")
    print(f"wrist_imgs shape: {wrist_imgs.shape}")

    obs_replay = {
        'qpos': traj_qpos,
        'qvel': traj_qvel,
        'images': {
            'left': left_imgs,
            'right': right_imgs,
            'wrist': wrist_imgs  # ✅ wrist 포함
        }
    }
    cfg['lang_intrs'] = raw_lang
    generate_h5(obs_replay, traj_actions, cfg, total_traj_cnt, act_root_dir_path, edit_flag)
    total_traj_cnt += 1



Loading TFDS dataset...


2025-08-08 12:07:42.772072: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-08 12:07:42.798124: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-08 12:07:42.798148: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-08 12:07:42.798930: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-08 12:07:42.803577: I tensorflow/core/platform/cpu_feature_guar

✅ TFDS 경로: /home/parkjeongsu/TinyVLA/Droid/droid_100
✅ 저장 경로: /home/parkjeongsu/TinyVLA/Droid/droid_with_lang/droid_1dot7t_lang_succ_t0001_s-0-0
raw_lang: close the lid of the box


  1%|          | 1/100 [00:00<00:47,  2.08it/s]

traj_len: 166
traj_actions shape: (166, 7)
traj_qpos shape: (166, 7)
traj_qvel shape: (166, 7)
left_imgs shape: (166, 180, 320, 3)
right_imgs shape: (166, 180, 320, 3)
wrist_imgs shape: (166, 180, 320, 3)
max_timesteps: 166
==== generate h5 ======
name: /observations/qpos, array: (166, 7)
name: /observations/qvel, array: (166, 7)
name: /action, array: (166, 7)
name: is_edited, array: ()
name: /observations/images/left, array: (166, 180, 320, 3)
name: /observations/images/right, array: (166, 180, 320, 3)
name: /observations/images/wrist, array: (166, 180, 320, 3)


...

 99%|█████████▉| 99/100 [00:16<00:00,  3.73it/s]

traj_len: 1120
traj_actions shape: (1120, 7)
traj_qpos shape: (1120, 7)
traj_qvel shape: (1120, 7)
left_imgs shape: (1120, 180, 320, 3)
right_imgs shape: (1120, 180, 320, 3)
wrist_imgs shape: (1120, 180, 320, 3)
max_timesteps: 1120
==== generate h5 ======
name: /observations/qpos, array: (1120, 7)
name: /observations/qvel, array: (1120, 7)
name: /action, array: (1120, 7)
name: is_edited, array: ()
name: /observations/images/left, array: (1120, 180, 320, 3)
name: /observations/images/right, array: (1120, 180, 320, 3)
name: /observations/images/wrist, array: (1120, 180, 320, 3)


100%|██████████| 100/100 [00:16<00:00,  6.19it/s]

No valid language instruction in this episode....


: 